## About Dataset
### Context
This is the sentiment140 dataset. It contains 1,600,000 tweets from twitter . The tweets have been annotated (0 = negative, 4 = positive) and they can be used to detect sentiment .

### Content
It contains the following 6 fields:

- target: the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)

- ids: The id of the tweet ( 2087)

- date: the date of the tweet (Sat May 16 23:58:44 UTC 2009)

- flag: The query (lyx). If there is no query, then this value is NO_QUERY.

- user: the user that tweeted (robotickilldozr)

- text: the text of the tweet (Lyx is cool)

In [81]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [82]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KIIT0001\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [83]:
# printing stopwords in English
print(stopwords.words('English'))
# stopwords do not hold much importance thus must be removed from the given text

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [84]:
df=pd.read_csv('training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')

In [85]:
# printing 1st 5 rows of the dataset
df.head(5)

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [86]:
# no. of rows and columns in dataset
df.shape

(1599999, 6)

We see the 1st row of the dataset being treated as the feature names.

In [87]:
column_names=['target','id','date','flag','user','text']
df=pd.read_csv('training.1600000.processed.noemoticon.csv',names=column_names,encoding='ISO-8859-1')

In [88]:
df.head(5)

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [89]:
df.shape

(1600000, 6)

In [90]:
df.isna().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [91]:
df['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [92]:
# converting the targer '4' to '1'
df.replace({'target':{4:1}},inplace=True)

In [93]:
df['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

- 0 --> Negative Tweets
- 1 --> Positive Tweets

## Stemming
- It is the process of reducing a word to its root word
- Eg: acting, actor, actress ---> act

In [94]:
pattern=re.compile('[^a-zA-Z]')

In [95]:
english_stopwords=stopwords.words('english')

In [96]:
port_stem=PorterStemmer()

In [97]:
def stemming(content): # content contains the text we pass from the dataset
    stemmed_content = re.sub(pattern,' ',content)  # removing all the characters from the content other than a-z or A-Z
    stemmed_content=stemmed_content.lower() # loweing all the characters from the content
    stemmed_content=stemmed_content.split() # splitting all the words from the content and storing it in a list
    stemmed_content=[port_stem.stem(word) for word in stemmed_content if word not in english_stopwords]# stemming each word presnt in the list and then putting back only those words which are not stopwords
    stemmed_content=' '.join(stemmed_content) # joining back all the words of the content together with a ' '

    return stemmed_content


In [98]:
df['stemmed_content']=df['text'].apply(stemming)

In [99]:
df.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [100]:
df['stemmed_content']

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object

## Independent and Dependent Variables

In [101]:
X=df['stemmed_content'].values
y=df['target'].values

In [102]:
X

array(['switchfoot http twitpic com zl awww bummer shoulda got david carr third day',
       'upset updat facebook text might cri result school today also blah',
       'kenichan dive mani time ball manag save rest go bound', ...,
       'readi mojo makeov ask detail',
       'happi th birthday boo alll time tupac amaru shakur',
       'happi charitytuesday thenspcc sparkschar speakinguph h'],
      dtype=object)

In [103]:
y

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

## Train Test Split

In [104]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,stratify=y,random_state=2)

In [105]:
X.shape,X_train.shape,X_test.shape

((1600000,), (1280000,), (320000,))

In [106]:
X_train

array(['watch saw iv drink lil wine', 'hatermagazin',
       'even though favourit drink think vodka coke wipe mind time think im gonna find new drink',
       ..., 'eager monday afternoon',
       'hope everyon mother great day wait hear guy store tomorrow',
       'love wake folger bad voic deeper'], dtype=object)

In [107]:
X_test

array(['mmangen fine much time chat twitter hubbi back summer amp tend domin free time',
       'ah may show w ruth kim amp geoffrey sanhueza',
       'ishatara mayb bay area thang dammit', ...,
       'destini nevertheless hooray member wonder safe trip', 'feel well',
       'supersandro thank'], dtype=object)

In [108]:
y_test[200]

1

## Vectorization

In [109]:
# converting the textual data into numerical values

vectorizer=TfidfVectorizer()

X_train_vec=vectorizer.fit_transform(X_train)
X_test_vec=vectorizer.transform(X_test)

In [110]:
print(X_train_vec)

  (0, 443066)	0.4484755317023172
  (0, 235045)	0.41996827700291095
  (0, 109306)	0.3753708587402299
  (0, 185193)	0.5277679060576009
  (0, 354543)	0.3588091611460021
  (0, 436713)	0.27259876264838384
  (1, 160636)	1.0
  (2, 288470)	0.16786949597862733
  (2, 132311)	0.2028971570399794
  (2, 150715)	0.18803850583207948
  (2, 178061)	0.1619010109445149
  (2, 409143)	0.15169282335109835
  (2, 266729)	0.24123230668976975
  (2, 443430)	0.3348599670252845
  (2, 77929)	0.31284080750346344
  (2, 433560)	0.3296595898028565
  (2, 406399)	0.32105459490875526
  (2, 129411)	0.29074192727957143
  (2, 407301)	0.18709338684973031
  (2, 124484)	0.1892155960801415
  (2, 109306)	0.4591176413728317
  (3, 172421)	0.37464146922154384
  (3, 411528)	0.27089772444087873
  (3, 388626)	0.3940776331458846
  (3, 56476)	0.5200465453608686
  :	:
  (1279996, 390130)	0.22064742191076112
  (1279996, 434014)	0.2718945052332447
  (1279996, 318303)	0.21254698865277746
  (1279996, 237899)	0.2236567560099234
  (1279996, 2910

In [111]:
print(X_test_vec)

  (0, 420984)	0.17915624523539803
  (0, 409143)	0.31430470598079707
  (0, 398906)	0.3491043873264267
  (0, 388348)	0.21985076072061738
  (0, 279082)	0.1782518010910344
  (0, 271016)	0.4535662391658828
  (0, 171378)	0.2805816206356073
  (0, 138164)	0.23688292264071403
  (0, 132364)	0.25525488955578596
  (0, 106069)	0.3655545001090455
  (0, 67828)	0.26800375270827315
  (0, 31168)	0.16247724180521766
  (0, 15110)	0.1719352837797837
  (1, 366203)	0.24595562404108307
  (1, 348135)	0.4739279595416274
  (1, 256777)	0.28751585696559306
  (1, 217562)	0.40288153995289894
  (1, 145393)	0.575262969264869
  (1, 15110)	0.211037449588008
  (1, 6463)	0.30733520460524466
  (2, 400621)	0.4317732461913093
  (2, 256834)	0.2564939661498776
  (2, 183312)	0.5892069252021465
  (2, 89448)	0.36340369428387626
  (2, 34401)	0.37916255084357414
  :	:
  (319994, 123278)	0.4530341382559843
  (319995, 444934)	0.3211092817599261
  (319995, 420984)	0.22631428606830145
  (319995, 416257)	0.23816465111736276
  (319995, 3

## Logistic Regression

In [112]:
model=LogisticRegression(max_iter=1000)

In [113]:
model.fit(X_train_vec,y_train)

LogisticRegression(max_iter=1000)

## Model Evaluation

### Accuracy Score

In [ ]:
y_train_pred=model.predict(X_train_vec)
training_data_accuracy=accuracy_score(y_train,y_train_pred)
print('Accuarcy on the training data:',training_data_accuracy)
# training data is also predicted to check for overfitting condition ----> which do not arise

Accuarcy on the training data: 0.81021328125


In [115]:
y_test_pred=model.predict(X_test_vec)
testing_data_accuracy=accuracy_score(y_test,y_test_pred)
print('Accuarcy on the testing data:',testing_data_accuracy)

Accuarcy on the testing data: 0.778003125


### Model Accuracy = 77.8%

## Saving the model using pickle

In [116]:
import pickle

In [117]:
pickle.dump(model,open('trained_model.pkl','wb'))

## Using the saved model for future predictions

In [118]:
# loading the saved model
loaded_model=pickle.load(open('trained_model.pkl','rb'))

In [ ]:
X_new=X_test_vec[200]
print(y_test[200])
print("TRUTH:")
if y_test[200]==0:
    print("NEGATIVE")
else:
    print("POSTIVE TWEET")
prediction=loaded_model.predict(X_new)
print(prediction[0])
print("PREDICTION:")
if prediction[0]==0:
    print("NEGATIVE TWEET")
else:
    print("POSITIVE TWEET")

1
TRUTH:
POSTIVE COMMENT/REVIEW
1
PREDICTION:
POSITIVE COMMENT/REVIEW


In [144]:
X_new=X_test_vec[3]
print(y_test[3])
print("TRUTH:")
if y_test[3]==0:
    print("NEGATIVE TWEET")
else:
    print("POSTIVE TWEET")
prediction=loaded_model.predict(X_new)
print(prediction[0])
print("PREDICTION:")
if prediction[0]==0:
    print("NEGATIVE TWEET")
else:
    print("POSITIVE TWEET")

0
TRUTH:
NEGATIVE TWEET
0
PREDICTION:
NEGATIVE TWEET


In [142]:
X_new=X_test_vec[201]
print(y_test[201])
print("TRUTH:")
if y_test[201]==0:
    print("NEGATIVE TWEET")
else:
    print("POSTIVE TWEET")
prediction=loaded_model.predict(X_new)
print(prediction[0])
print("PREDICTION:")
if prediction[0]==0:
    print("NEGATIVE TWEET")
else:
    print("POSITIVE TWEET")

# this shows the margin of error

1
TRUTH:
POSTIVE TWEET
0
PREDICTION:
NEGATIVE TWEET
